In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pytz
from pytz import common_timezones, all_timezones
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from datetime import datetime

http://greenteapress.com/wp/think-bayes/

Roger Labbe has transformed _Think Bayes_ into IPython notebooks where you can modify and run the code

There are several excellent modules for doing Bayesian statistics in Python, including pymc and OpenBUGS. I chose not to use them for this book because you need a fair amount of background knowledge to get started with these modules, and I want to keep the prerequisites minimal. If you know Python and a little bit about probability, you are ready to start this book.

## Chapter 1 Bayes's Theorem

### Conditional probability

The usual notation for conditional probability is p(A|B), which is the probability of A given that B is true.

### Conjoint probability

In general, the probability of a conjunction is

p(A  and  B) = p(A) p(B|A) 

### The cookie problem

We’ll get to Bayes’s theorem soon, but I want to motivate it with an example called the cookie problem.1 Suppose there are two bowls of cookies. Bowl 1 contains 30 vanilla cookies and 10 chocolate cookies. Bowl 2 contains 20 of each.
Now suppose you choose one of the bowls at random and, without looking, select a cookie at random. The cookie is vanilla. What is the probability that it came from Bowl 1?

This is a conditional probability; we want p(Bowl 1 | vanilla), but it is not obvious how to compute it. If I asked a different question—the probability of a vanilla cookie given Bowl 1—it would be easy:

p(vanilla | Bowl 1) = 3/4 
Sadly, p(A|B) is not the same as p(B|A), but there is a way to get from one to the other: Bayes’s theorem.

### Bayes's theorem

$
p(A|B) = \frac{ p(A) p(B|A)}{p(B)}
$ 

And that’s Bayes’s theorem! It might not look like much, but it turns out to be surprisingly powerful.

$
p(B_1|V) = \frac{(1/2) (3/4)}{ (1/2)(3/4) + (1/2)(1/2)} = 3/5
$

So the vanilla cookie is evidence in favor of the hypothesis that we chose Bowl 1, because vanilla cookies are more likely to come from Bowl 1.

This example demonstrates one use of Bayes’s theorem: it provides a strategy to get from p(B|A) to p(A|B). This strategy is useful in cases, like the cookie problem, where it is easier to compute the terms on the right side of Bayes’s theorem than the term on the left.

### The diachronic interpretation

There is another way to think of Bayes’s theorem: it gives us a way to update the probability of a hypothesis, H, in light of some body of data, D.
This way of thinking about Bayes’s theorem is called the diachronic interpretation. “Diachronic” means that something is happening over time; in this case the probability of the hypotheses changes, over time, as we see new data.

Rewriting Bayes’s theorem with H and D yields:

$
p(H|D) = \frac{p(H)p(D|H)}{p(D)}
$

In this interpretation, each term has a name:

* p(H) is the probability of the hypothesis before we see the data, called the prior probability, or just **prior**.
* p(H|D) is what we want to compute, the probability of the hypothesis after we see the data, called the **posterior**.
* p(D|H) is the probability of the data under the hypothesis, called the **likelihood**.
* p(D) is the probability of the data under any hypothesis, called the **normalizing constant**.

Sometimes we can compute the prior based on background information. For example, the cookie problem specifies that we choose a bowl at random with equal probability.
In other cases the prior is subjective; that is, reasonable people might disagree, either because they use different background information or because they interpret the same information differently.

The likelihood is usually the easiest part to compute. In the cookie problem, if we know which bowl the cookie came from, we find the probability of a vanilla cookie by counting.

The normalizing constant can be tricky. It is supposed to be the probability of seeing the data under any hypothesis at all, but in the most general case it is hard to nail down what that means.

Most often we simplify things by specifying a set of hypotheses that are

**Mutually exclusive**:
At most one hypothesis in the set can be true, and
**Collectively exhaustive**:
There are no other possibilities; at least one of the hypotheses has to be true.
I use the word suite for a set of hypotheses that has these properties.


In the cookie problem, there are only two hypotheses—the cookie came from Bowl 1 or Bowl 2—and they are mutually exclusive and collectively exhaustive.

In that case we can compute p(D) using the law of total probability, which says that if there are two exclusive ways that something might happen, you can add up the probabilities like this:

$
p(D) = p(B1) p(D|B1) + p(B2) p(D|B2) 
$


Plugging in the values from the cookie problem, we have


$
p(D) = (1/2) (3/4) + (1/2) (1/2) = 5/8 
$

### The M&M problem

M&M’s are small candy-coated chocolates that come in a variety of colors. Mars, Inc., which makes M&M’s, changes the mixture of colors from time to time.
In 1995, they introduced blue M&M’s. Before then, the color mix in a bag of plain M&M’s was 30% Brown, 20% Yellow, 20% Red, 10% Green, 10% Orange, 10% Tan. Afterward it was 24% Blue , 20% Green, 16% Orange, 14% Yellow, 13% Red, 13% Brown.

Suppose a friend of mine has two bags of M&M’s, and he tells me that one is from 1994 and one from 1996. He won’t tell me which is which, but he gives me one M&M from each bag. One is yellow and one is green. What is the probability that the yellow one came from the 1994 bag?

This problem is similar to the cookie problem, with the twist that I draw one sample from each bowl/bag. This problem also gives me a chance to demonstrate the table method, which is useful for solving problems like this on paper. In the next chapter we will solve them computationally.

The first step is to enumerate the hypotheses. The bag the yellow M&M came from I’ll call Bag 1; I’ll call the other Bag 2. So the hypotheses are:

* A: Bag 1 is from 1994, which implies that Bag 2 is from 1996.
* B: Bag 1 is from 1996 and Bag 2 is from 1994.

Now we construct a table with a row for each hypothesis and a column for each term in Bayes's theorem:
 
| | p(H) | p(D &#124; H)  | p(H)p(D &#124; H) | p(H &#124; D) | 
|:-----:|:--------------:|:-----------------:|:-------------:|:---------------------:|
|A| 1/2 | (20)(20) | 200 |  20/27     |
|B| 1/2 | (14)(10)  | 70   |   7/27 |   

The second column has the likelihoods, which follow from the information in the problem. For example, if A is true, the yellow M&M came from the 1994 bag with probability 20%, and the green came from the 1996 bag with probability 20%. If B is true, the yellow M&M came from the 1996 bag with probability 14%, and the green came from the 1994 bag with probability 10%. Because the selections are independent, we get the conjoint probability by multiplying.


The third column is just the product of the previous two. The sum of this column, 270, is the normalizing constant. To get the last column, which contains the posteriors, we divide the third column by the normalizing constant.

That’s it. Simple, right?


Well, you might be bothered by one detail. I write p(D|H) in terms of percentages, not probabilities, which means it is off by a factor of 10,000. But that cancels out when we divide through by the normalizing constant, so it doesn’t affect the result.

When the set of hypotheses is mutually exclusive and collectively exhaustive, you can multiply the likelihoods by any factor, if it is convenient, as long as you apply the same factor to the entire column.